In [ ]:
from datascience import *
from prob140 import *     
import numpy as np

In [ ]:
# Run this code

import itertools as it
from collections import Counter

def clean_string(string):
    """
    Cleans an input string by replacing all newlines with spaces, and then
    removing all letters not in *allowable_letters*
    """
    string = string.replace("\n"," ")
    return "".join([i for i in string.lower().strip() if i in allowable_letters])

def load_bigrams(text):
    """
    Takes a string which has already been cleaned, and returns a dictionary
    of conditional bigram probabilities
    
    cond_bigram[(a,b)] = P(X_{n+1} = b | X_{n} = a)
    
    Uses Laplace smoothing with size $1$, to remove zero transition probabilities
    """
    bigram_counter = Counter(list(it.product(allowable_letters,repeat=2)))
    gram_counter = Counter(allowable_letters*len(allowable_letters))
    for l1,l2 in zip(text,text[1:]):
        bigram_counter[(l1,l2)] += 1
        gram_counter[l1] += 1
    cond_bigram = {k:v/gram_counter[k[0]] for k,v in bigram_counter.items()}
    return cond_bigram
    
def bigram_from_file(filename):
    """
    Given a filename, this reads it, cleans it, and returns the conditional bigram
    """
    file_text = open(filename).read()
    file_text = clean_string(file_text)
    return load_bigrams(file_text)

def reverse_cipher(cipher):
    return {v:k for k,v in cipher.items()}

def print_differences(cipher1,cipher2):
    for k in cipher1:
        if cipher1[k] != cipher2[k]:
            print("%s: %s  %s"%(k,cipher1[k],cipher2[k]))
            
def num_errors(cipher,encoded_text,original_text):
    decoded = np.array(list(decode_text(encoded_text,cipher)))
    original = np.array(list(original_text))
    num_errors = np.count_nonzero(decoded != original)
    return num_errors

# Lab 6: Code Breaking by MCMC #

In today's lab, we will be breaking codes encrypted using a cipher

To see how this works, please read [Sections 14.3 (Code Breaking)](https://textbook.prob140.org/ch14/Code_Breaking.html) and [14.4 (Markov Chain Monte Carlo)](https://textbook.prob140.org/ch14/Markov_Chain_Monte_Carlo.html) of the Course Notes. The lab follows those sections closely.

The text that we are going to decode was written in English. For data on the frequencies of all the different bigrams in English, we will start by counting all the bigrams in *War and Peace* by Tolstoy. That is one of the longest novels in English (actually in Russian, but we are using an English translation from Project Gutenberg) and is often used as a "corpus" or body of language on which to base analyses of other text.

Please start by running all the cells above this one.

## Alphabet and Bigrams #
To keep the calculations manageable, we will restrict ourselves to a 27-character alphabet, consisiting of the 26 lower case letters and a space. The cell below places all 27 in a list.

In [ ]:
allowable_letters = list("abcdefghijklmnopqrstuvwxyz ")

Next, we find the relative frequencies of all bigrams in War and Peace, and place them in the dictionary `wp_bigrams`. You don't need to know what a dictionary is; it's fine to imagine it containing the same information as a Table that has a column with the bigrams and another column with the relative frequencies. 

In [ ]:
wp_bigrams = bigram_from_file("warandpeace.txt")

### Question 1
As in class, the model is that the sequence of characters in the text is a Markov Chain. The state space of the chain is the alphabet of 27 characters. Of course the text has other punctuation, but we are stripping those, and we are replacing upper case letters by lower case.

Construct the transition matrix for this Markov Chain based on the relative frequencies in `wp_bigrams` defined above.

We have started the code off for you by defining the transition function `wp_transition`. You may find `allowable_letter` helpful

In [ ]:
def wp_transition(a,b):
    if (a,b) in wp_bigrams:
        return wp_bigrams[(a,b)]
    return 0

wp_bigram_mc = ...
wp_bigram_mc

## Decoders

As you saw in class, a decoder is a permutation of the alphabet. We apply this permutation to our encoded text in order to generate the decoded text.

For this lab, we will be representing the decoder as a Python dictionary. Here is an example of how a dictionary works, using a three-letter alphabet 'a', 'b', 'c'.

Recall from class that if the decoder is ['b', 'c', 'a'] then the substitutions for decoding the encrypted message are

$$
a \to b ~~~~~~~~ b \to c ~~~~~~~~ c \to a
$$

This decoder written as a dictionary dictionary looks like this:

    simple_decoder = {'a':'b','b':'c','c':a'}

To access a value in the decoder, you can use the method `.get`. For example, to access the translation of *a*, use
    
    simple_decoder.get('a')

#### Keeping Spaces Intact ####
We will assume that our substitution code keeps spaces unchanged: neither do spaces get coded as letters, nor do letters get coded as spaces. Of course letters will precede and follow spaces, but spaces will be fixed points in the encoding permutation. 

So our decoder must keep spaces unchanged as well. The following cell defines a function called `random_decoder` which will randomly generate a decoder. It will operate on the 26 letters, omitting the space.

The function `random_decoder` simply constructs a random permutation of the alphabet and places it in a dictionary to make it a decoder.

In [ ]:
decoder_letters = np.array(list("abcdefghijklmnopqrstuvwxyz")) # We don't operate on spaces

identity_decoder = {letter:letter for letter in decoder_letters}

# Random starting decoder
def random_decoder():
    """ Random decoder """
    new_letters = decoder_letters.copy()
    np.random.shuffle(new_letters)
    return {orig:new for orig,new in zip(decoder_letters,new_letters)}


### Question 2

Define a function `decode_text` that takes as its arguments a string to be decoded and the decoder. It should return the decoded string.

Remember that we are only decoding alphabetical characters. If a character is not in the decoder (e.g. a space), leave the character alone.

In [ ]:

def decode_text(string, decoder):
    new_string = ""
    
    for char in string:
        if char in decoder:
            new_letter = ...
        else:
            new_letter = ...
        
        # Now we append the letter to the back of the new string
        new_string = new_string + new_letter
        
    return new_string

## Metropolis Algorithm: Proposal Chain ##
The Metropolis algorithm we will use is essentially the same as the one defined in class and proved to converge to the correct distribution. There is one tweak to increase computatational accuracy, and another to deal with spaces. You shouldn't worry about either of them.

Please refer to Section 14.4 of the Course Notes for the notation being used below.

- The state space $S$ is the set of all 26! possible decoders, that is, all 26! possible permutations of letters.

- The "proposal" transition matrix $Q$ is chosen as follows. Given that the chain is at decoder $i$, pick another decoder by randomly swapping two elements of the code. This is the transition matrix of an irreducible chain; you can get from any decoder to any other by a sequence of switches. Also, $Q$ is symmetric. For any two decoders $i$ and $j$,

$$
Q(i, j) = \frac{1}{\binom{26}{2}}, ~~~~ \text{if } i \text{ and } j \text{ differ in two places}
$$
and $Q(i, j) = 0$ otherwise.

### Question 3

In the following cell, define the function `generate_proposed_decoder` which takes an existing decoder and returns a new decoder as follows: it randomly picks two different letters of the alphabet and swaps the decoding for those letters.

For example, if we have the decoder `{'a':'b','b':'c','c':a'}` and decide to swap `a` and `b`, our new decoder should be `{'a':'c','b':'b','c':a'}`. 

In [ ]:
def generate_proposed_decoder(decoder):
    new_decoder = decoder.copy()
    
    
    letter1 = ...
    letter2 = ...
    
    new_value_of_letter1 = ...
    new_value_of_letter2 = ...
    
    # This code replaces the value of letter1 and letter2 with new_value_of_letter1 and new_value_of_letter2
    new_decoder[letter1] = new_value_of_letter1
    new_decoder[letter2] = new_value_of_letter2
    return new_decoder


## The Target Distribution $\pi$ ##

For any two characters $x_1$ and $x_2$, let $B(x_1, x_2)$ be the $(x_1, x_2)$th entry in the bigram transition matrix. Suppose the decoded text consists of the characters $x_0, x_1, \ldots , x_n$. Recall from class that we defined the *score* of the corresponding decoder $i$ to be

$$
s(i) = \prod_{k=0}^{n-1}B(x_k, x_{k+1})
$$

For decoder $i$, we would like it to appear among our randomly selected decoders with target probability $\pi(i)$, where $\pi(i)$ is the likelihood of $i$ given the data. As we saw in class, the likelihood $\pi(i)$ is proportional to $s(i)$ but we don't know the constant of proportionality. However, for any two decoders $i$ and $j$, we can calculate

$$
\frac{\pi(j)}{\pi(i)} = \frac{s(j)}{s(i)}
$$

Here's a tweak: When the string is long, the score for any decoder is going to be very small. So we will work with the log-score instead; that is, with the $\log$ of the score. 

$$
\log(s(i)) = \sum_{k=0}^{n-1} \log(B(x_k, x_{k+1}))
$$


### Question 4

Define the function `log_probability_mc` which takes a string and a bigram markov chain and finds the log probability of the "path" that follows those letters.

Don't use `np.log(mc.prob_of_path)` because the probabilities will round too soon and you will have problems with numerical inaccuracy. Instead, you can use the function [`mc.log_prob_of_path`](https://probability.gitlab.io/prob140/html/_autosummary/prob140.MarkovChain.log_prob_of_path.html#prob140.MarkovChain.log_prob_of_path). This takes as its first argument the starting character of the string, and as its next argument a list or array containing the characters in the rest of the string.

You may find the following operations from last week's lab helpful:

* string[0] returns the first letter of a string. For example:
```python
>>> x = 'HELLO'
>>> x[0]
'H'
```
* string[1:] returns everything except the first letter of a string. For example:
```python
>>> x[1:]
'ELLO'
```
* list(string) splits the string into a list of its characters.

In [ ]:
def log_probability_mc(string, bigram_mc):
    start = ...
    
    path = list(...)
    
    return bigram_mc.log_prob_of_path(start, path)

### Question 5

Define a function `log_probability_decoder_string` as follows.

- Its arguments are an encrypted string, a decoder, and a bigram transition matrix.
- It decodes the encrypted string using the decoder, and returns the log score of the decoder.

You will find that the functions `decode_text` and `log_probability_mc` that you wrote are useful here.

In [ ]:
def log_probability_decoder_string(string,decoder,bigrams):
    return ...

### The Acceptance Ratios ###
The Metropolis algorithm creates a new chain with $\pi$ as its limit distribution. Recall that if new chain is at $i$, then the the decision about whether or not to accept a transition to a new state $j$ is made by looking at the ratios $\pi(j)/\pi(i)$. We saw that this ratio was the same as $s(j)/s(i)$. Since we are working with log scores, it is worth noting that

$$
\frac{s(j)}{s(i)} < 1 ~~~ \iff ~~~ s(j) < s(i) ~~~ \iff ~~~
\log(s(j)) < \log(s(i)) 
$$

because $\log$ is an increasing function.

Also, to save you some possible mystification in one of the code cells below, observe that

$$
\frac{s(j)}{s(i)} = e^{\log(\frac{s(j)}{s(i)})} = e^{\log(s(j)) - \log(s(i))}
$$

The reason for these calculations is that we are working with log scores, not scores. So every statement that we made in terms of scores has to be rewritten in terms of log scores.

## Implementing the Metropolis Algorithm ##

### Question 6

Define the function `metropolis` which will take as its arguments:

- the encrypted text, as a string
- the transition matrix of bigrams
- the number of repetitions for the Metropolis algorithm. 

The function returns the decoder that it arrives at after performing the Metropolis algorithm the specified number of times. If the number of repetitions is large, you know from class that the distribution of this random decoder is close to the desired distribution $\pi$ which is the likelihood distribution of the decoder given the data. Therefore you expect to end up with a decoder that has a high likelihood compared to the other decoders.

Between iterations, you should keep track of the following variables:

1. `bestDecoder`: the decoder that has the highest log score
2. `bestScore`: the log score associated with the bestDecoder
3. `decoder`: the decoder that you are currently working with
4. `lastScore`: the log score of the current decoder

In each iteration, you should do the following:

1. Generate a new decoder based on the current decoder; the "new" decoder might be the same as the current one.
2. Calculate the log score of your new decoder.
3. **Important:** Follow the Metropolis algorithm to decide whether or not to move to a new decoder.
4. If the new decoder's log score is the best seen so far, update `bestScore` and `bestDecoder`

In [ ]:
import time

def p_coin(p):
    """
    Flips a coin that comes up heads with probability p
    and returns 1 if Heads, 0 if Tails
    """
    return np.random.random() < p


def metropolis(string_to_decode, bigrams, reps):
    decoder = random_decoder() # Starting decoder
    
    bestDecoder = decoder
    bestScore = log_probability_decoder_string(string_to_decode,bestDecoder,bigrams)
    
    lastScore = bestScore
    
    for rep in range(reps):
        
        # This will print out our progress
        if rep*10%reps == 0: # Repeat every 10%
            time.sleep(.01)
            decoded_text = decode_text(string_to_decode,bestDecoder)[:40]
            print('Score: %.00f \t Guess: %s'%(bestScore,decoded_text))
            
        #########################
        # Your code starts here #
        #########################
        proposed_decoder = ...
        log_s_orig = ...
        log_s_new = ...
        
        if log_s_new > log_s_orig or p_coin(np.e**(log_s_new-log_s_orig)): 
            # If better than before or p-coin flip works
            
            ...
            
            if log_s_new > bestScore:
                ...
    return bestDecoder

## Running It ##

The text you are going to first encrypt and then decode is taken from [Winston Churchill's speech](https://en.wikipedia.org/wiki/We_shall_fight_on_the_beaches) to the House of Commons on 4 June 1940. Naturally, it contains upper case letters and punctuation that we have not included in our alphabet. In the cell below, the function `clean_string` strips out all the punctuation and replaces upper case letters by the corresponding lower case letters.

In [ ]:
ogtext = """
I have, myself, full confidence that if all do their duty, if nothing is neglected, 
and if the best arrangements are made, as they are being made, we shall prove ourselves 
once again able to defend our Island home, to ride out the storm of war, and to outlive 
the menace of tyranny, if necessary for years, if necessary alone. At any rate, that is 
what we are going to try to do. That is the resolve of His Majesty’s Government-every man 
of them. That is the will of Parliament and the nation. 
"""
original_string = clean_string(ogtext)
encoding_cipher = random_decoder()
string_to_decode = decode_text(original_string,encoding_cipher)
string_to_decode


Run the following cell to generate your decoder

In [ ]:
new_decoder = metropolis(string_to_decode, wp_bigram_mc, 10000) 

Make sure the text above matches the original text. Run the cell below to decode the entire string

In [ ]:
decode_text(string_to_decode, new_decoder)

### Question 7

Decode `secret_text` using MCMC.

In [ ]:
secret_text = 'uc rwl yjce wudrjgs jv rwl bjgyq jcys m vlb elclgmrujcd wmzl tllc egmcrlq rwl gjyl jv qlvlcquce vgllqji uc urd wjng jv imauini qmcelg u qj cjr dwgucf vgji rwud gldpjcdutuyurs u blykjil ur u qj cjr tlyulzl rwmr mcs jv nd bjnyq lakwmcel pymkld burw mcs jrwlg pljpyl jg mcs jrwlg elclgmrujc rwl lclges rwl vmurw rwl qlzjrujc bwukw bl tguce rj rwud lcqlmzjg buyy yuewr jng kjncrgs mcq myy bwj dlgzl ur mcq rwl eyjb vgji rwmr vugl kmc rgnys yuewr rwl bjgyq mcq dj is vlyyjb milgukmcd mdf cjr bwmr sjng kjncrgs kmc qj vjg sjn mdf bwmr sjn kmc qj vjg sjng kjncrgs is vlyyjb kuruolcd jv rwl bjgyq mdf cjr bwmr milgukm buyy qj vjg sjn tnr bwmr rjelrwlg bl kmc qj vjg rwl vgllqji jv imc'

### (optional)

Who is the secret text attributed to?

In [ ]:
_ = autograder.grade('q1')

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [autograder.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]

In [ ]:
import gsExport
gsExport.generateSubmission()